# Global data
Data source: Compustat Global - Fundamentals Annual

Date range: 1987-06 - 2023-08 

Identifiers:
- datadate
- Global Company Key (gvkey)
- International Security ID (ISIN)
- FYEAR -- Data Year - Fiscal (FYEAR)
- SEDOL (SEDOL)
- CONML -- Company Legal Name (CONML)
- FIC -- ISO Country Code - Incorporation (FIC)
- CITY -- City (CITY)
- SIC -- Standard Industry Classification Code (SIC)

In [5]:
import pandas as pd
import numpy as np
import json
country_code = "global"

In [6]:
def winsorize_series(s: pd.Series, limits=[0.05, 0.95], msk=None) -> pd.Series:
    s = s.copy()
    q = s.quantile(limits)
    if isinstance(q, pd.Series) and len(q) == 2:
        if msk is None:
            s[s < q.iloc[0]] = q.iloc[0]
            s[s > q.iloc[1]] = q.iloc[1]
        else:
            s[s < q.iloc[0]] = msk
            s[s > q.iloc[1]] = msk
    return s

In [7]:
# load data
df = pd.read_csv(
    "./data/hmji7l6oih5ry4a5.csv").dropna(subset=["datadate", "fyear", "isin"])
df.rename(columns={"fyear": "year"}, inplace=True)
df

,gvkey,year,datadate,at,capx,ceq,che,dlc,dltt,ebitda,...,xrd,isin,sedol,cshoi,nicon,tstkni,fic,city,conml,sic
0,1166,1996.0,1996-12-31,611.980,81.439,178.075,70.609,88.641,87.975,132.496,...,57.599,NL0000334118,5165294,33.058,68.402,0.0,NLD,Almere,ASM International NV,3559.0
1,1166,1997.0,1997-12-31,724.115,77.507,35.302,73.164,266.221,102.718,-72.618,...,85.944,NL0000334118,5165294,33.379,-155.308,0.0,NLD,Almere,ASM International NV,3559.0
2,1166,1998.0,1998-12-31,623.539,53.202,39.370,41.908,144.215,189.325,78.947,...,79.945,NL0000334118,5165294,34.541,1.247,0.0,NLD,Almere,ASM International NV,3559.0
3,1166,1999.0,1999-12-31,425.035,30.587,65.552,19.862,111.896,7.997,63.842,...,47.145,NL0000334118,5165294,40.108,11.099,0.0,NLD,Almere,ASM International NV,3559.0
4,1166,2000.0,2000-12-31,777.940,71.366,308.322,106.810,44.620,31.660,248.290,...,73.800,NL0000334118,5165294,48.797,94.272,0.0,NLD,Almere,ASM International NV,3559.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872975,358077,2020.0,2021-03-31,255.826,3.353,153.294,1.049,21.004,5.900,19.686,...,NaN,INE08S201019,BRJ51Y5,NaN,NaN,NaN,IND,Ahmedabad,Yasons Chemex Care Limited,2860.0
872976,358077,2021.0,2022-03-31,277.928,0.004,172.857,0.570,21.628,6.512,20.057,...,NaN,INE08S201019,BRJ51Y5,NaN,NaN,NaN,IND,Ahmedabad,Yasons Chemex Care Limited,2860.0
872977,358077,2022.0,2023-03-31,275.859,0.014,207.204,0.844,26.903,7.737,12.189,...,NaN,INE08S201019,BRJ51Y5,NaN,NaN,NaN,IND,Ahmedabad,Yasons Chemex Care Limited,2860.0
872978,358078,2021.0,2021-12-31,0.175,NaN,0.114,0.175,0.000,0.000,-0.358,...,NaN,GB00BMH3KM70,BMH3KM7,NaN,NaN,NaN,GBR,London,Metals One PLC,1000.0


In [8]:
market_data = pd.read_csv("./data/nnswdeirjo2zhlef.csv")
# keep year and month
df["datadate"] = pd.to_datetime(df["datadate"]).dt.strftime("%Y-%m")
market_data["datadate"] = pd.to_datetime(
    market_data["datadate"]).dt.strftime("%Y-%m")
market_data

,gvkey,datadate,prccm,isin
0,5,2004-01,10488.07,NaN
1,5,2004-02,10583.92,NaN
2,5,2004-03,10357.70,NaN
3,5,2004-04,10225.57,NaN
4,5,2004-05,10188.45,NaN
...,...,...,...,...
9325584,357438,2023-07,25.00,NaN
9325585,357452,2023-06,32.60,CNE1000064F5
9325586,357452,2023-07,32.15,CNE1000064F5
9325587,357455,2023-06,129.00,ID1000190705


In [9]:
# merge data
df = pd.merge(df, market_data, on=["isin", "datadate"], how="left")
df

,gvkey_x,year,datadate,at,capx,ceq,che,dlc,dltt,ebitda,...,sedol,cshoi,nicon,tstkni,fic,city,conml,sic,gvkey_y,prccm
0,1166,1996.0,1996-12,611.980,81.439,178.075,70.609,88.641,87.975,132.496,...,5165294,33.058,68.402,0.0,NLD,Almere,ASM International NV,3559.0,NaN,NaN
1,1166,1997.0,1997-12,724.115,77.507,35.302,73.164,266.221,102.718,-72.618,...,5165294,33.379,-155.308,0.0,NLD,Almere,ASM International NV,3559.0,NaN,NaN
2,1166,1998.0,1998-12,623.539,53.202,39.370,41.908,144.215,189.325,78.947,...,5165294,34.541,1.247,0.0,NLD,Almere,ASM International NV,3559.0,NaN,NaN
3,1166,1999.0,1999-12,425.035,30.587,65.552,19.862,111.896,7.997,63.842,...,5165294,40.108,11.099,0.0,NLD,Almere,ASM International NV,3559.0,NaN,NaN
4,1166,2000.0,2000-12,777.940,71.366,308.322,106.810,44.620,31.660,248.290,...,5165294,48.797,94.272,0.0,NLD,Almere,ASM International NV,3559.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861889,358077,2020.0,2021-03,255.826,3.353,153.294,1.049,21.004,5.900,19.686,...,BRJ51Y5,NaN,NaN,NaN,IND,Ahmedabad,Yasons Chemex Care Limited,2860.0,NaN,NaN
861890,358077,2021.0,2022-03,277.928,0.004,172.857,0.570,21.628,6.512,20.057,...,BRJ51Y5,NaN,NaN,NaN,IND,Ahmedabad,Yasons Chemex Care Limited,2860.0,NaN,NaN
861891,358077,2022.0,2023-03,275.859,0.014,207.204,0.844,26.903,7.737,12.189,...,BRJ51Y5,NaN,NaN,NaN,IND,Ahmedabad,Yasons Chemex Care Limited,2860.0,NaN,NaN
861892,358078,2021.0,2021-12,0.175,NaN,0.114,0.175,0.000,0.000,-0.358,...,BMH3KM7,NaN,NaN,NaN,GBR,London,Metals One PLC,1000.0,NaN,NaN


In [10]:
# drop duplicates
df = df.sort_values(by=["isin", "datadate"], ascending=[False, True])
del df["datadate"]
df = df.drop_duplicates(subset=["isin", "year"], keep="first")
df

,gvkey_x,year,at,capx,ceq,che,dlc,dltt,ebitda,ppent,...,sedol,cshoi,nicon,tstkni,fic,city,conml,sic,gvkey_y,prccm
257371,214221,1996.0,NaN,11.047,NaN,NaN,NaN,NaN,NaN,NaN,...,6752468,NaN,NaN,NaN,ZWE,NaN,Rothmans Rhodesia (Zimbabwe),NaN,NaN,NaN
257372,214221,1997.0,NaN,10.312,NaN,NaN,NaN,NaN,NaN,NaN,...,6752468,NaN,NaN,NaN,ZWE,NaN,Rothmans Rhodesia (Zimbabwe),NaN,NaN,NaN
257061,214041,2002.0,91245.827,4219.170,50726.385,1153.013,246.664,0.000,-7024.208,82831.200,...,BQPDKZ0,80.000,-3541.765,NaN,ZWE,Harare,Khayah Cement Ltd,3241.0,NaN,NaN
257062,214041,2003.0,117754.659,4295.273,58585.823,13893.523,673.580,15823.350,23490.485,72565.142,...,BQPDKZ0,80.000,9357.796,NaN,ZWE,Harare,Khayah Cement Ltd,3241.0,NaN,NaN
257063,214041,2009.0,37.782,1.724,17.568,1.997,1.627,0.218,2.184,26.677,...,BQPDKZ0,80.000,2.689,NaN,ZWE,Harare,Khayah Cement Ltd,3241.0,214041.0,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390484,251250,2010.0,286215.888,NaN,33655.770,NaN,4032.828,18559.357,8811.433,2336.860,...,6313898,5557.775,2077.508,NaN,ARE,Dubai,Emirates Bank International Ltd,6020.0,NaN,NaN
390485,251250,2011.0,284613.386,NaN,34934.774,NaN,13515.547,8017.153,7497.366,2576.990,...,6313898,5557.775,2269.023,NaN,ARE,Dubai,Emirates Bank International Ltd,6020.0,NaN,NaN
390486,251250,2012.0,308296.351,NaN,36452.307,NaN,6789.039,16437.792,6627.167,2469.156,...,6313898,5557.775,2291.730,NaN,ARE,Dubai,Emirates Bank International Ltd,6020.0,NaN,NaN
390487,251250,2013.0,342061.275,NaN,41710.787,NaN,5318.725,19743.681,6836.186,2757.869,...,6313898,5557.775,2870.861,NaN,ARE,Dubai,Emirates Bank International Ltd,6020.0,NaN,NaN


In [11]:
import os
if not os.path.exists('output'):
    os.makedirs('output')

1. Capex $_{t}$ / AT $_{t-1}$

In [12]:
df["CapitalExpenditures/TotalAsset"] = df["capx"] / df["at"] * 100
df["CapitalExpenditures/TotalAsset"] = winsorize_series(
    df["CapitalExpenditures/TotalAsset"], limits=[0.01, 0.99], msk=None)
df[["isin", "year", "CapitalExpenditures/TotalAsset"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.00000,2017.000000,2023.000000
CapitalExpenditures/TotalAsset,609240.0,5.333774,6.685076,0.009086,1.119699,3.09181,6.788758,38.652907


In [13]:
result_json = df[["isin", "year", "CapitalExpenditures/TotalAsset"
                 ]].to_json(orient='split', index=False)
result_dict = json.loads(result_json)

In [14]:
output_dict = {
    "varibale_name": "CapitalExpenditures/TotalAsset",
    "variable_description": "Capital Expenditures (CAPX) / Total Assets (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [15]:
with open("./output/"+country_code+"_capexatratio.json", "w") as file:
    json.dump(output_dict, file, indent=4)

2. Capex $_{t}$ / PPENT $_{t-1}$

In [16]:
df["CapitalExpenditures/PropertyPlant&Equipment"] = df["capx"] / df["ppent"] * 100
# inf -> nan
df["CapitalExpenditures/PropertyPlant&Equipment"] = df["CapitalExpenditures/PropertyPlant&Equipment"].replace(np.inf, np.nan)
df["CapitalExpenditures/PropertyPlant&Equipment"] = winsorize_series(df["CapitalExpenditures/PropertyPlant&Equipment"],
                                     limits=[0.01, 0.99], msk=None)
df[["isin", "year", "CapitalExpenditures/PropertyPlant&Equipment"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.000000,2023.000000
CapitalExpenditures/PropertyPlant&Equipment,606485.0,24.559542,32.663941,0.079367,6.572027,14.887434,29.368664,231.889231


In [17]:
result_json = df[["isin", "year", "CapitalExpenditures/PropertyPlant&Equipment"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [18]:
output_dict = {
    "varibale_name": "CapitalExpenditures/PropertyPlant&Equipment",
    "variable_description": "Capital Expenditures (CAPX) / Property, Plant and Equipment - Total (Net) (PPENT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [19]:
with open("./output/"+country_code+"_capexppentratio.json", "w") as file:
    json.dump(output_dict, file, indent=4)

3. PPENT $_{t}$ / AT $_{t}$

In [20]:
df["PropertyPlant&Equipment/TotalAssets"] = df["ppent"] / df["at"] * 100
df["PropertyPlant&Equipment/TotalAssets"] = winsorize_series(df["PropertyPlant&Equipment/TotalAssets"],
                                                            limits=[0.01, 0.99], msk=None)
df[["isin", "year", "PropertyPlant&Equipment/TotalAssets"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.0,2004.000000,2011.000000,2017.000000,2023.000000
PropertyPlant&Equipment/TotalAssets,798444.0,27.036857,23.562303,0.0,5.763464,22.411424,42.210686,90.259301


In [21]:
result_json = df[["isin", "year", "PropertyPlant&Equipment/TotalAssets"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [22]:
output_dict = {
    "varibale_name": "PropertyPlant&Equipment/TotalAssets",
    "variable_description": "Capital Expenditures (CAPX) / Property, Plant and Equipment - Total (Net) (PPENT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [23]:
with open("./output/"+country_code+"_ppeatratio.json", "w") as file:
    json.dump(output_dict, file, indent=4)

4. CASH $_{t}$ / AT $_{t - 1}$

In [24]:
df["Chsh/TotalAssets"] = df["che"] / df["at"] * 100
df["Chsh/TotalAssets"] = winsorize_series(df["Chsh/TotalAssets"],
                                          limits=[0.01, 0.99], msk=None)
df[["isin", "year", "Chsh/TotalAssets"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.000000,2023.000000
Chsh/TotalAssets,696826.0,16.886015,17.919215,0.020183,4.059354,11.018077,23.180885,86.901547


In [25]:
result_json = df[["isin", "year", "Chsh/TotalAssets"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [26]:
output_dict = {
    "varibale_name": "Chsh/TotalAssets",
    "variable_description": "Cash and Short-Term Investments (CHE) / Total Assets (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [27]:
with open("./output/"+country_code+"_cashatratio.json", "w") as file:
    json.dump(output_dict, file, indent=4)

5. Cost of Debt: XINT $_{t}$ / (DLTT $_{t}$ + DLC $_{t}$ )

In [28]:
df["CostofDebt"] = df["xint"] / (df["dltt"] + df["dlc"]) * 100
# inf -> nan
df["CostofDebt"] = df["CostofDebt"].replace(np.inf, np.nan)
df["CostofDebt"] = winsorize_series(df["CostofDebt"],
                                    limits=[0.01, 0.99], msk=None)
df[["isin", "year", "CostofDebt"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.000000,2023.000000
CostofDebt,664660.0,11.401841,27.620155,0.014138,2.840909,5.435899,9.394359,227.992576


In [29]:
result_json = df[["isin", "year", "CostofDebt"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [30]:
output_dict = {
    "varibale_name": "CostofDebt",
    "variable_description": "Interest and Related Expense - Total (XINT) / (Long-Term Debt - Total (DLTT) + Debt in Current Liabilities - Total (DLC)). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [31]:
with open("./output/"+country_code+"_costofdebt.json", "w") as file:
    json.dump(output_dict, file, indent=4)

6. RD $_{t}$ / AT $_{t-1}$

In [32]:
df["R&D/TotalAssets"] = df["xrd"] / df["at"] * 100
# inf -> nan
df["R&D/TotalAssets"] = df["R&D/TotalAssets"].replace(np.inf, np.nan)
df["R&D/TotalAssets"] = winsorize_series(df["R&D/TotalAssets"],
                                    limits=[0.01, 0.99], msk=None)
df[["isin", "year", "R&D/TotalAssets"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.000000,2023.000000
R&D/TotalAssets,233727.0,3.995619,7.557544,0.003332,0.416889,1.591796,3.927168,50.945505


In [33]:
result_json = df[["isin", "year", "R&D/TotalAssets"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [34]:
output_dict = {
    "varibale_name": "R&D/TotalAssets",
    "variable_description": "Research and Development Expense (XRD) / Assets - Total (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [35]:
with open("./output/"+country_code+"_rdatratio.json", "w") as file:
     json.dump(output_dict, file, indent=4)

7. ROA: NI $_{t}$ / AT $_{t-1}$

In [36]:
df["ROA"] = df["nicon"] / df["at"] * 100
df["ROA"] = winsorize_series(df["ROA"],
                                    limits=[0.01, 0.99], msk=None)
df[["isin", "year", "ROA"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.00000,2023.000000
ROA,714813.0,-1.313232,22.524152,-152.065938,-0.647079,2.443004,6.36332,35.234818


In [37]:
result_json = df[["isin", "year", "ROA"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [38]:
output_dict = {
    "varibale_name": "ROA",
    "variable_description": "Net Income (Loss) - Consolidated (NICON) / Assets - Total (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [39]:
with open("./output/"+country_code+"_roa.json", "w") as file:
    json.dump(output_dict, file, indent=4)

8. TobinQ (AT $_{t}$ + MKTVAL $_{t}$ - CEQ $_{t}$ ) / AT $_{t}$ 

In [40]:
# 用股价计算MKTVAL，这里只能算出普通股总价值
df["mktval"] = df["prccm"] * (df["cshoi"] - df["tstkni"])
df[["isin", "year", "mktval"]].describe().T
# 待验证

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999e+00,1987.00000,2004.00000,2011.0000,2017.000,2.023000e+03
mktval,202437.0,559430.890815,5.125353e+07,-147501.55728,724.26104,4892.6499,35577.714,1.743594e+10


In [41]:
df["TobinQ"] = (df["at"] + df["mktval"] - df["ceq"]) / df["at"]
df["TobinQ"] = winsorize_series(df["TobinQ"],
                                limits=[0.01, 0.99], msk=None)
df[["isin", "year", "TobinQ"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.0000,2023.000000
TobinQ,202391.0,1.482714,1.281589,0.437888,0.879123,1.073054,1.5543,9.344818


In [42]:
result_json = df[["isin", "year", "TobinQ"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [43]:
output_dict = {
    "varibale_name": "TobinQ",
    "variable_description": "(Assets - Total (AT) + (Par Value - Issue (PV) * (Com Shares Outstanding - Issue (CSHOI) - Treasury Stock - Number of Common Shares - Issue (TSTKNI))) - Common/Ordinary Equity - Total (CEQ)) / Assets - Total (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [44]:
with open("./output/"+country_code+"_tobinq.json", "w") as file:
    json.dump(output_dict, file, indent=4)

9. Size: Log(AT $_{t}$ )

In [45]:
df["Size"] = np.log(df["at"])
# -inf -> nan
df["Size"] = df["Size"].replace(-np.inf, np.nan)
df["Size"] = winsorize_series(df["Size"],
                                limits=[0.01, 0.99], msk=None)
df[["isin", "year", "Size"]].describe().T

/home/djh592/dev/python/datacollection/venv/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/djh592/dev/python/datacollection/venv/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.000000,2023.000000
Size,850041.0,7.382636,3.307937,0.043059,5.075074,7.198531,9.549238,15.842146


In [46]:
result_json = df[["isin", "year", "Size"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [47]:
output_dict = {
    "varibale_name": "Size",
    "variable_description": "Log(Assets - Total (AT)). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [48]:
with open("./output/"+country_code+"_size.json", "w") as file:
    json.dump(output_dict, file, indent=4)

10. Leverage: (DLTT $_{t}$ + DLC $_{t}$ ) / (AT $_{t-1}$ )

In [49]:
df["Leverage"] = (df["dltt"]+df["dlc"]) / df["at"]*100
df["Leverage"] = winsorize_series(df["Leverage"],
                              limits=[0.01, 0.99], msk=None)
df[["isin", "year", "Leverage"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.0,2004.000000,2011.000000,2017.000000,2023.000000
Leverage,784421.0,23.500442,22.150880,0.0,4.563205,19.252875,35.935231,112.595575


In [50]:
result_json = df[["isin", "year", "Leverage"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [51]:
output_dict = {
    "varibale_name": "Leverage",
    "variable_description": "(Long-Term Debt - Total (DLTT) + Debt in Current Liabilities - Total (DLC)) / Assets - Total (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [52]:
with open("./output/"+country_code+"_leverage.json", "w") as file:
    json.dump(output_dict, file, indent=4)

11. EBITDA: ETITDA $_{t}$ / AT $_{t}$

In [53]:
df["EBITDA"] = df["ebitda"] / df["at"]*100
df["EBITDA"] = winsorize_series(df["EBITDA"],
                              limits=[0.01, 0.99], msk=None)
df[["isin", "year", "EBITDA"]].describe().T

,count,mean,std,min,25%,50%,75%,max
year,858976.0,2010.152494,8.129999,1987.000000,2004.000000,2011.000000,2017.000000,2023.00000
EBITDA,846994.0,4.791028,20.350376,-123.815185,1.906434,6.968589,12.536644,50.49951


In [54]:
result_json = df[["isin", "year", "EBITDA"
                  ]].to_json(orient="split", index=False)
result_dict = json.loads(result_json)

In [55]:
output_dict = {
    "varibale_name": "EBITDA",
    "variable_description": "Earnings Before Interest (EBITDA) / Assets - Total (AT). Winsorized at 1\% and 99\% level.",
    "data_source": "Compustat Global - Fundamentals Annual",
    "reference": "",
    "identifier_entity": "isin",
    "identifier_time": "year",
}
output_dict["data"] = result_dict["data"]

In [56]:
with open("./output/"+country_code+"_ebitda.json", "w") as file:
    json.dump(output_dict, file, indent=4)